This notebook compares the scores of the local models and the model chain

In [1]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from predict_proba import Chain
import os

In [2]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [3]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [4]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'EDSS-after_2y', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'NRELAP', 'CESEV']

Note: once we obtain the best ordering, change the order here!

In [5]:
# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before
0,46.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1,19.35,NaN,58.0,51.0,3.90,36.0,1.25,0.047619,2.75,0.148148,0.125000,0.833333,0.730769,0.0,0.0,0.0,0.0
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1,23.80,21.5,31.5,NaN,6.15,NaN,NaN,NaN,4.50,0.481481,0.166667,0.728571,0.658228,1.0,0.0,0.0,NaN


In [6]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [7]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
EDSS-after_2y       float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
NRELAP               object
CESEV                object
dtype: object

In [8]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [9]:
# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

features_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

Is it a problem that not all folds have the exact same number?

In [16]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

---

### Local Models

In [10]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS 
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [11]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [12]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [13]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.90 (± 0.03)
NRELAP_1.0: 1.01 (± 0.03)
NRELAP_2.0: 1.04 (± 0.03)
NRELAP_3.0: 1.10 (± 0.06)
NRELAP_4+: 1.04 (± 0.03)
CESEV_MILD: 1.40 (± 0.10)
CESEV_MODERATE: 2.00 (± 0.11)
CESEV_SEVERE: 1.42 (± 0.13)


In [14]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.02 (± 0.08)
CESEV: 1.61 (± 0.30)


In [15]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [17]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (local):
KFSS_M-2y: 0.19 (± 0.02)
KFSS_P-2y: 0.26 (± 0.03)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.29 (± 0.08)
NHPT-2y: 0.43 (± 0.16)
P_R36-SF12-after: 0.32 (± 0.05)
M_R36-SF12-after: 0.44 (± 0.03)
SES_after: 0.32 (± 0.04)
SLEC_after: 0.36 (± 0.05)
EDSS-after_2y: 0.26 (± 0.04)
KFSS_M-after_2y: 0.35 (± 0.03)
KFSS_P-after_2y: 0.50 (± 0.06)


In [18]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19372533, 0.25511106, 0.11932071, 0.29263897, 0.42918357,
       0.31580612, 0.44241826, 0.31571977, 0.35626238, 0.2573924 ,
       0.34984472, 0.50361086, 1.01910429, 1.60795844])

In [19]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02298089, 0.02515065, 0.01410889, 0.07763896, 0.16422008,
       0.04671572, 0.02705947, 0.04196047, 0.04506327, 0.03614755,
       0.02702441, 0.06397672, 0.07774342, 0.30242559])

In [20]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.4612926333741412


In [21]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.38039058129521375


### Propagate predictions

In [22]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [23]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [24]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [25]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.90 (± 0.02)
NRELAP_1.0: 1.02 (± 0.03)
NRELAP_2.0: 1.04 (± 0.03)
NRELAP_3.0: 1.09 (± 0.07)
NRELAP_4+: 1.04 (± 0.04)
CESEV_MILD: 1.41 (± 0.09)
CESEV_MODERATE: 2.01 (± 0.10)
CESEV_SEVERE: 1.43 (± 0.13)


In [26]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.02 (± 0.08)
CESEV: 1.61 (± 0.30)


In [27]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]


# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - propagate predictions):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - propagate predictions):
KFSS_M-2y: 0.19 (± 0.02)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.02)
T25FW-2y: 0.28 (± 0.07)
NHPT-2y: 0.41 (± 0.17)
P_R36-SF12-after: 0.32 (± 0.05)
M_R36-SF12-after: 0.46 (± 0.02)
SES_after: 0.31 (± 0.04)
SLEC_after: 0.35 (± 0.03)
EDSS-after_2y: 0.26 (± 0.04)
KFSS_M-after_2y: 0.37 (± 0.03)
KFSS_P-after_2y: 0.49 (± 0.07)


In [28]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19372533, 0.25460378, 0.12279878, 0.27915699, 0.41223603,
       0.31539661, 0.45883228, 0.30661281, 0.35426623, 0.26484844,
       0.36528968, 0.49268812, 1.01822731, 1.61244086])

In [29]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02298089, 0.02724153, 0.01835027, 0.0650433 , 0.16795926,
       0.04617155, 0.01939952, 0.04452414, 0.03192497, 0.04214704,
       0.02666003, 0.07127051, 0.07599077, 0.29854153])

In [30]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.46079451835570323


In [31]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.37686325876892834


### Propagate true values

In [52]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [53]:
yi_test_dummies_list[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0
13,1,0,0,0,0,0,1,0
20,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2449,1,0,0,0,0,0,0,0
2454,1,0,0,0,0,0,0,0
2457,1,0,0,0,0,0,0,0
2458,1,0,0,0,0,0,0,0


In [54]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [55]:
yi_test_dummies_avg[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
0,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
3,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
8,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
13,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
20,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
...,...,...,...,...,...,...,...,...
2449,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2454,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2457,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2458,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058


In [56]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [57]:
concatenated_dfs[4]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
6,0.484695,0.243972,0.171333,0.070000,0.030000,0.272500,0.587500,0.140000
9,0.417810,0.259810,0.212827,0.070000,0.039553,0.286846,0.563154,0.150000
10,0.550000,0.320000,0.080000,0.030000,0.020000,0.100000,0.830000,0.070000
16,0.477551,0.220716,0.196188,0.075545,0.030000,0.289520,0.525385,0.185095
17,0.610000,0.210000,0.100000,0.030000,0.050000,0.090000,0.807500,0.102500
...,...,...,...,...,...,...,...,...
2438,0.450000,0.330000,0.130000,0.070000,0.020000,0.180000,0.750000,0.070000
2443,0.700000,0.200000,0.060000,0.020000,0.020000,0.180000,0.740000,0.080000
2451,0.740000,0.110000,0.060000,0.070000,0.020000,0.380000,0.520000,0.100000
2453,0.600000,0.180000,0.140000,0.050000,0.030000,0.140000,0.730000,0.130000


In [58]:
yi_test_dummies_list[0]['NRELAP_0.0']

0       1
3       1
8       1
13      1
20      0
       ..
2449    1
2454    1
2457    1
2458    1
2460    1
Name: NRELAP_0.0, Length: 500, dtype: int32

In [64]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.90 (± 0.03)
NRELAP_1.0: 0.98 (± 0.02)
NRELAP_2.0: 1.00 (± 0.03)
NRELAP_3.0: 1.02 (± 0.04)
NRELAP_4+: 1.02 (± 0.05)
CESEV_MILD: 1.24 (± 0.06)
CESEV_MODERATE: 1.98 (± 0.12)
CESEV_SEVERE: 1.26 (± 0.10)


In [65]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 0.98 (± 0.06)
CESEV: 1.49 (± 0.36)


In [66]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [67]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

In [68]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.19 (± 0.02)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.02)
T25FW-2y: 0.29 (± 0.08)
NHPT-2y: 0.41 (± 0.18)
P_R36-SF12-after: 0.31 (± 0.05)
M_R36-SF12-after: 0.45 (± 0.02)
SES_after: 0.31 (± 0.05)
SLEC_after: 0.35 (± 0.03)
EDSS-after_2y: 0.25 (± 0.03)
KFSS_M-after_2y: 0.36 (± 0.03)
KFSS_P-after_2y: 0.48 (± 0.05)


In [69]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19372533, 0.25484864, 0.11739269, 0.28632101, 0.40594059,
       0.31014775, 0.44519679, 0.3126169 , 0.34824118, 0.24934822,
       0.35799463, 0.47889038, 0.98185138, 1.49028132])

In [70]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02298089, 0.02694025, 0.01840422, 0.07888323, 0.18267041,
       0.04662568, 0.01623856, 0.05075839, 0.03481199, 0.03397452,
       0.03200176, 0.05240171, 0.0560189 , 0.35758062])

In [40]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.44519977054246623


In [71]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.42817745157463233
